# 준비작업

ToDo List
1. __발급불가 판정 나는 거 pass하는 코드__
1. 사건번호로 채무자키 찾을 때 관할법원/관련사건법원 까지 함께 살피기(여러개 나오는 경우 공백으로 두기 때문에 수작업을 해야할 뿐 잘못된 키가 불려오지는 않을 것)
2. 1을 위해서 관할법원 정식명칭으로 맵핑하는 작업(일단은 지방> 지방법원 만 적용중. 현재 광주 아니면 부산이라 문제 없음)
3. 신규등록해야 하는 건 선별해서 법조치에, 기등록건은 법조치키를 법비용에
4. 대상자구분 확인하기
5. (전계좌, 부분계좌, 단일계좌 확인해야....)
6. 법조치 기등록건 구분 및 법조치키 불러오기

### import
- 이행권고 출력 및 판결문 목록에 등재 (화해권고는 다른 사람이 출력중)
- 승계는 수작업(발급당사자는 원고만, 납부인)

In [2]:
import pyautogui as au
import time
import pandas as pd
import re
from os.path import join
import os
import numpy as np
from tqdm import tqdm
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
#from selenium.webdriver.common.alert import Alert
#from selenium.common.exceptions import NoAlertPresentException, NoSuchWindowException # NoSuchElementException, UnexpectedAlertPresentException, WebDriverException,
import random
from file_function import read_filelist
from datetime import date
# from datetime import timedelta
# import traceback
# 정렬순서 : 확정일자(내림), 법원(올림), 사건번호(내림)

dtype = {'채무자키':str, '계좌키':str, '가상계좌번호':str, '법조치키':str}

### 함수, 전역변수

In [ ]:
##################################
# 사건번호 눌러보고 집행문 발급 가능한 것만으로 목록작성 하도록!!!!!!!!!
lastday = "250211" # '집행문신청_' 다음의 접미사로 꼭 6자리 날짜가 아니어도 상관없음
법원인증서="헬프자산관리대부[주]()001102620170619111002243" # 인증서 여러개고, 법원인증서가 첫번째가 아닌 경우 지정해줘야 함

이행권고출력 = False # 이행권고 출력할지 말지 (240513이후 확정건부터 박근혜과장님이 출력) 이행권고는 판결정본만 출력(송달후 2주내 이의제기 없어야 확정되므로 송달및확정증명원 필요 없음)
집행문부여출력 = True # 이행권고와 집행문부여의 판결정본 출력 과정은 똑같다. 집행문부여는 확정,송달증명서 출력해야
보조pc = False
수동발급여부 = False # 예 or 아니요
##################################
cols_idx = [1,2,5,7,8,9,10]

if 보조pc :
    excel_dir = join(r"\\192.168.10.83\전자소송")
else : 
    excel_dir = join(r"D:\4.관리\전자소송")
excel_file = "집행문신청_" + lastday
path_excel = join(excel_dir, excel_file+".xlsx")
bank = "신한은행"
driver, wait = None, None


class ErrorMsg(Exception):
    def __init__(self, message):
        super().__init__(message)
        self.message = message

    def __str__(self):
        return self.message


def 파일명(폴더:str, 키워드:str) :
    file_list = os.listdir(폴더)
    account_filename = [file for file in file_list if re.search(키워드, file)]
    if len(account_filename) == 1 :
        return join(폴더,account_filename[0])
    else :
        print("해당 키워드가 들어간 파일이 하나가 아닙니다.")

# pause
def lwt() : time.sleep(2.5)
def mwt() : time.sleep(1)
def swt() : time.sleep(random.uniform(0.2,0.5))

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def con_scourt() :
    driver = set_chrome_driver()
    # driver.get("https://ecfs.scourt.go.kr/psp/index.on?m=PSPA2GM01") # 제증명발급신청. 메인화면 하면 로그인 오류 생김
    driver.get("https://ecfs.scourt.go.kr/psp/index.on")
    
    driver.maximize_window() # 최대화 필수(상단 메뉴 보였다 안 보였다...)
    wait = WebDriverWait(driver, 30) 
    # 로그인 버튼
    # wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_PSPMAMP01_wframe_btn_lgn'))).click() # 통합로그인 클릭
    wait.until(EC.element_to_be_clickable((By.ID, 'mf_pmf_content1_btnMainLogin'))).click() # 로그인 클릭
    mwt()
    return driver, wait

def certify_esc() :
    wait.until(EC.element_to_be_clickable((By.ID, "xwup_CancelButton"))).click() # 취소버튼 클릭
    
def certify() :
    # 오류 나면 기다리지 말고 취소 후 다시 하도록 wait 일부러 안 씀
    lwt()
    elem = driver.find_element(By.ID, "xwup_certselect_tek_input1")
    swt()
    
    # 여러 인증서중 법원용 인증서 선택
    cert_elem_xpath = '//*[@id="xwup_cert_table"]/table/tbody/tr/td/div[text()="' + 법원인증서 + '"]'
    cert_elem = driver.find_element(By.XPATH, cert_elem_xpath)
    cert_elem.click()
    
    swt()
    elem.send_keys("help8700765!")
    swt()
    driver.find_element(By.ID, 'xwup_OkButton').click() # 인증서 확인
    mwt()

def login() : 
    
    # 아이디 입력란 찾기 및 입력
    input_element = driver.find_element("id", "mf_pfwork_ibx_elpUserIdForCert")
    input_element.send_keys("help2255")
    swt()
    
    # 로그인 버튼 클릭
    login_button = driver.find_element("id", "mf_pfwork_btn_certlogin")
    login_button.click()
    mwt()
    
    # 로그인(인증서)
    certify()
    lwt()

# ############ 공통 사용 #############

def 대상자명분리(피고:str) : 
    match = re.match(r"^(.*)(?: 외)(.*)", 피고)
    return match.group(1) if match else 피고
    

def close_all_popups_and_refresh(driver):
    try:
        # 모든 팝업 창 닫기
        if len(driver.window_handles) > 1 : 
            for handle in driver.window_handles[1:]:
                driver.switch_to.window(handle)
                driver.close()
        
        # 첫 번째 창으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])
        
    except Exception as e:
        print(f"팝업 창을 닫는 동안 오류가 발생했습니다: {e}")

    # 브라우저 새로고침
    driver.refresh()
    lwt()


def 제증명발급() :
    try :
        wait = WebDriverWait(driver, 5)
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_depth1_menu3'))).click() # 각종신청
        swt()
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_anc_menuid_150702'))).click() # 제증명발급신청
        swt()
    except : # 상단 날개 안 먹는 경우 새로고침
        close_all_popups_and_refresh()
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_depth1_menu3'))).click() # 각종신청
        swt()
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_anc_menuid_150702'))).click() # 제증명발급신청
        swt()
        
    
def 이행권고() : # 제증명 > 발급
    wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_depth1_menu5'))).click() # 상단메뉴, 나의 전자소송
    swt()
    driver.find_element(By.ID, 'mf_pfheader_anc_menuid_150701').send_keys(Keys.ENTER) # 송달문서 정(등)본발급
    swt()

def 오류메시지처리(driver) : 
    # 팝업 내부에서 메시지 요소 찾기
    message_element = driver.find_element(By.CLASS_NAME, "w2textbox")
    popup_message = message_element.text

    if re.match("인터넷으로는 제증명을 신청할 수 없는", popup_message) : # 인터넷 출력 불가
        return "인터넷출력불가"
    elif re.search("수동발급으로 신청하시겠습니까",popup_message) : # 수동발급 팝업임
        return "수동발급"
    elif re.search('위임장 또는 당사자임을 소명하는 자료', popup_message) : # 타지점 신청건
        return "타지점"
    elif re.search('당사자별 자동발급으로 신청하시겠습니까', popup_message) :
        return "자동발급"
    else : 
        return "기타오류"
    
def 오류확인버튼클릭(popup) : 
    wait = WebDriverWait(popup, 3)  # 최대 10초 대기
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'confirm'))).click()
    # popup.find_element(By.CLASS_NAME,'confirm').click() # 오류 확인 버튼 클릭
    mwt()
    
def 문서제출버튼클릭() :
    wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_btn_docmtSbmsn'))).click() # 문서제출   
    mwt()

def 예아니요버튼클릭(응답:str) :
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f".w2trigger.btn_cm.pop[value='{응답}']"))).click() # 예
    mwt()

def 완료버튼클릭() : 
    wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_btn_sendLst'))).click() # 완료버튼
    swt()
    
   
def 오류체크() :
    """오류(작업 대상이 아닌 예상치 못한 알람) 발생시 확인 버튼 눌러준 후 ErrorMsg(오류종류) 일으킴"""
    popups = driver.find_elements(By.CLASS_NAME, "w2popup_window") # 요소 없어도 에러 안나게 s는 리스트 반환
    if not popups : # 오류없는 경우
        return None
    popup = popups[0] # popup는 리스트임
    오류종류 = 오류메시지처리(popup) 
    
    # '예' 또는 '아니오' 버튼 클릭하는 경우
    if 오류종류 == "수동발급" :  
        button_value = "예" if 수동발급여부 else "아니요"
    elif 오류종류 == "자동발급" : # 전에는 기본값이었던 것
        button_value = "예"
        
    # 알림 : 확인버튼 누르는 경우
    else :  
        오류확인버튼클릭(popup)
        raise ErrorMsg(오류종류)
    
    예아니요버튼클릭(button_value)
    
    return None
    
def select_court_event(court, event, name="", 발급관련=False) :
    """법원 및 사건번호 입력"""
    
    # 법원
    if 발급관련 :
        법원목록_id = "mf_pfwork_sbx_cortList" 
        사건구분_id = "mf_pfwork_acp_csDvs_input"
        사건일련번호_id = "mf_pfwork_ibx_csNo"
    else :
        법원목록_id = "mf_pfwork_sbx_cortCd"
        사건구분_id = "mf_pfwork_acp_csDvsCd_input"
        사건일련번호_id = "mf_pfwork_ibx_csSerial"
    Select(driver.find_element(By.ID, 법원목록_id)).select_by_visible_text(court)     
    swt() 
    
    event = re.search("(\d{4,4})([가-힣]+)(\d+)", event)
    year = event[1]
    sign = event[2]
    sn = event[3]
    
    # 사건번호 선택
    Select(driver.find_element(By.ID, 'mf_pfwork_sbx_csYear')).select_by_visible_text(year) # 연도 선택
    element = driver.find_element(By.ID, 사건구분_id) # 사건구분 선택
    element.clear() 
    element.send_keys(sign)
    swt()

    element = driver.find_element(By.ID, 사건일련번호_id) # 사건일련번호 선택
    element.clear()
    element.send_keys(sn)
    swt()

    # 당사자명 선택
    if name == "" : 
        pass
    else : 
        대상자 = 대상자명분리(name)
        element = driver.find_element(By.ID, 'mf_pfwork_ibx_btprtNmShow') 
        element.clear()
        element.send_keys(대상자)
        mwt()
    
# ########### 제증명 신청 관련 #################
def 증명서선택함수(docu_kind : str) :
    """
    집행문,송달,확정,판결정본 중 선택하기
    docu_kind : 집행문:0 | 송달:1 | 확정:2 | 판결정본:3
    """
    if docu_kind =="집행문" : docu_kind_id = "mf_pfwork_gen_area_0_gen_docDtl_0_spn_doc"
    elif docu_kind =="송달" : docu_kind_id = "mf_pfwork_gen_area_2_gen_docDtl_0_spn_doc"
    elif docu_kind =="확정" : docu_kind_id = "mf_pfwork_gen_area_2_gen_docDtl_1_spn_doc"
    elif docu_kind =="판결정본" : docu_kind_id = "mf_pfwork_gen_area_1_gen_docDtl_0_spn_doc"
    
    try : 
        wait = WebDriverWait(driver, 5) 
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_depth1_menu3'))).click() # 각종신청
        swt()
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_anc_menuid_130201'))).click() # 제증명발급신청
    except : # 상단 날개 메뉴 인식 안 되는 경우 refresh 후 재작업
        close_all_popups_and_refresh()
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_depth1_menu3'))).click() # 각종신청
        swt()
        wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_anc_menuid_130201'))).click() # 제증명발급신청
        
   
    # 증명서 종류 선택
    wait.until(EC.element_to_be_clickable((By.ID, docu_kind_id))).click()
    mwt()
   

# # 수정 ----------------------------------------------------------------------------------------------------------------------------------------
def 문서작성함수(which, 문서종류) : ##############
    # 양수금, 대여금, 신용카드이용대금 인지
    target_event_name = "양수금|대여금|신용카드이용대금|집행문부여"
    element = wait.until(EC.visibility_of_element_located((By.ID, 'mf_pfwork_wfm_csBas_csNm')))
    swt()
    eventname = element.text
    if re.search(target_event_name, element.text) == None : # 타겟 사건명이 아니면
        return False, eventname
    else :
        try : 
            # 집행문, 송달증명원
            if which == 0:
                wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_wfm_issuBtprt_btn_issuBtprtChc'))).click() # 발급대상자자선택 버튼
                mwt()
                
                # 송달과 집행문의 등록 버튼이 다름
                if 문서종류 == "송달" : 
                    reg_id = 'mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_btn_csNtznBtprtRegister'
                    allchk_id = "mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_grd_csNtznBtprtChcList_chkAll"
                    tbl_id = "mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_grd_csNtznBtprtList_body_tbody"
                elif 문서종류 == "집행문" : 
                    reg_id = 'mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_btn_issuBtprtRegister'
                    allchk_id = "mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_grd_issuBtprtChcList_chkAll"
                    tbl_id = "mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_grd_issuBtprtList_body_tbody"
                
                tbl = driver.find_element(By.XPATH, f'//tbody[@id="{tbl_id}"]/tr')
                
                rows = tbl.find_elements(By.XPATH, '//tr[td/span[text()="피고"]]')

                for row in rows:
                    checkbox = row.find_element(By.XPATH, './/td/input[@type="checkbox"]')
                    if not checkbox.is_selected():
                        checkbox.click()  # 체크박스 선택
                swt()
                
                
                # 증명원 할 때는 됐던 코드
                # # 팝업창 컨트롤(발급당사자)
                # checkboxes = driver.find_elements(By.XPATH, "//table[@id='mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_grd_csNtznBtprtList_body_table']//tr[td[@data-col_id='btprtDvsCdNm' and normalize-space()='피고']]//input[@type='checkbox']")

                # for checkbox in checkboxes:
                #     if not checkbox.is_selected():
                #         checkbox.click()
                
                
                driver.find_element(By.ID, reg_id).click() # 등록
                mwt()
                driver.find_element(By.ID, allchk_id).click()  # 선택된 당사자 모두 선택
                swt()
                driver.find_element(By.ID, 'mf_pfwork_wfm_issuBtprt_PSPC7PP01_wframe_btn_confirm').click() # 확인
                mwt()   
                return True, eventname
                
            # 확정증명원
            elif which == 1 :
                wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_wfm_aplyInf_btn_findNtzn'))).click() # 공증항목 선택 버튼
                mwt()
                
                # 팝업창 컨트롤(발급당사자)
                driver.find_element(By.ID, 'G_mf_pfwork_wfm_aplyInf_PSPC7PP02_wframe_grd_csNtznList___radio_rdo_0').click() # 단추선택
                driver.find_element(By.ID, 'mf_pfwork_wfm_aplyInf_PSPC7PP02_wframe_btn_confirm').click() # 확인
                swt()
        except : 
            return False, eventname
            

def check_document() : #############
    time.sleep(7) # pdf문서 대기
    wait.until(EC.presence_of_element_located((By.ID, 'viewer')))
    
    # 문서 이상없음 체크 및 확인버튼
    driver.find_element(By.ID, 'mf_pfwork_cbx_confirm_input_0').click() # 문서이상없음
    driver.find_element(By.ID, 'mf_pfwork_btn_done').click() # 확인완료
    mwt()

    
# ########## print ############
def confirm_print() :   # 게시물에 맞는 발급버튼 누르기
    while(True) :
        win_print = au.getWindowsWithTitle("인쇄")
        if win_print == [] :
            time.sleep(3)
        else :
            break
    # 창 활성화
    if not win_print[0].isActive:
        win_print[0].activate()
        
    time.sleep(0.2)
    au.press('enter')

def print_execution(docu_kind) :
    "사건번호로 조회 후 집행문(정본포함)/송달증명/확정증명 발급"
    
    try :
        wait = WebDriverWait(driver, 5) # 기본은 30초로 너무 기니 여기서는 제한한다.
        # kind_docu의 발급 버튼 누르기
        발급버튼_path = f'//tbody[@id="mf_pfwork_grid_crtfcsIssu_body_tbody"]//a[text()="{docu_kind}"]/ancestor::tr//td[8]//button'
        wait.until(EC.element_to_be_clickable((By.XPATH, 발급버튼_path))).click()
    except :
        return driver.find_element(By.XPATH, 발급버튼_path).text
    
    swt()
    wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_grid_dicIssu_button_0_3'))).click() # 종이출력버튼
    예아니요버튼클릭("예")
    time.sleep(7) # 인쇄창 대기 5초에서 7초로 늘림 240514
    
    # 인쇄ok버튼
    confirm_print() 
    
    # 확인버튼클릭
    lwt()
    오류확인버튼클릭(driver)
    
    # 뒤로가기 버튼
    if docu_kind != "확정증명" :
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mf_pfwork_btn_prev"]'))).click() 
    mwt()
    return "발급"


def print_이행권고() : 
    # 이건 판결정본이 항상 첫번째행에 있음을 가정한 경로. 
    # 만약 다른 사례가 발견되면, "class title이 판결정본인 td에 다음다음 td"로 수정
    # cell다음의 숫자는 행번호 열번호임(0_4 : 첫행 다섯번째열)
    결정문발급버튼 = driver.find_element(By.XPATH, '//*[@id="mf_pfwork_grd_issu_cell_0_4"]//button')
    버튼이름 = re.sub(" ","", 결정문발급버튼.text)
    if 버튼이름 == '발급' : 
        swt()
        결정문발급버튼.click()
    else : return 버튼이름
    
    예아니요버튼클릭("예")
    time.sleep(7) # 인쇄창 대기
    
    # 인쇄 확인 버튼
    confirm_print()
    
    # 확인
    mwt()
    오류확인버튼클릭(driver)
    return 버튼이름
        
# ############## 통합 함수 ##############
# # 수정 ----------------------------------------------------------------------------------------------------------------------------------------
def fix_delivery(docu_kind, v) :
    """확정증명 or 송달증명 신청 종합 함수 """
    
    # 메뉴선택
    증명서선택함수(docu_kind) # 제증명신청, 제증명 종류
    
    # 사건선택 및 오류체크
    select_court_event(v.법원, v.사건번호, v.피고) # 법원 사건번호
    driver.find_element(By.ID, 'mf_pfwork_btn_confirm').click() # 확인 버튼
    mwt()
    오류체크()
    
    # 문서작성
    mwt()
    which = 1 if docu_kind == "확정" else 0
    문서작성함수(which, docu_kind) # 사건 or 발급당사자선택
    
    driver.find_element(By.ID, "mf_pfwork_btn_wrtCmptn").click() # 작성완료
    mwt()
    check_document() # 문서확인
    
    mwt()
    문서제출버튼클릭() 
    mwt()
    certify_esc()
    mwt()
    문서제출버튼클릭() 
    mwt()
    
    try :
        certify() # 인증서 
    except :
        driver.find_element(By.ID, 'xwup_CancelButton').click() # 취소
        mwt()
        문서제출버튼클릭()
        mwt()
        certify()
    
    lwt() # 제출 후 인증은 lwt()추가!
    
    완료버튼클릭()
    
def 출력_사건번호조회(법원, 사건번호) :
    # 사건번호로 조회
    제증명발급() 
    driver.find_element(By.ID, "mf_pfwork_rad_choice_input_1").click() # 사건번호 radio
    swt()
    select_court_event(법원, 사건번호, name="", 발급관련=True)
    driver.find_element(By.ID, "mf_pfwork_btn_search").click() # 조회버튼 클릭
    mwt()
    
# def 소송비용납부_au():
#     au.click(693, 804)
#     swt()
#     au.press('tab')
#     swt
#     au.press('down')
#     mwt()
#     au.press('tab')
#     mwt()
#     au.press('tab')
#     mwt()
#     au.press('enter')
#     mwt()
#     # 납부하시겠습니까? (예:탭한번)
#     au.press('tab')
#     swt()
#     au.press('enter')
#     mwt()
#     wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_btn_docmtSbmsn'))).click() # 문서제출
#     swt()
    
# def login_au() : 
#     # 로그인(id) # 셀레니움 안 돼서.. 
#     au.click(801,432)
#     swt()
#     au.press('tab')
#     swt()
#     au.write("help")
#     au.press("num2")
#     au.press("num2")
#     au.write("55")
#     swt()
#     au.press('tab')
#     swt()
#     au.press('enter')
    
#     # 로그인(인증서)
#     certify()
#     lwt()
    
    

### 엑셀 열기 - 키 연결해서 채무상태 확인 후 진행

In [8]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
print(len(df))
df.iloc[[0,-1], cols_idx]

12


,법원,사건번호,확정일자,피고,집행문,확정,송달
0,광주지방법원,2024가소612043,2025.02.08,남성일,,,
11,광주지방법원,2023가소590337,2024.06.08,강문수,,,


### df에 채무자키 계좌키 제소신청일 추가하기

In [9]:
# 최근 dataset pkl 에서 법조치 불러오기
##########################################
last_dataset_day = '250131'
###########################################
if 보조pc : 
    fn_law = 파일명(join(r'\\192.168.0.75\자산관리팀\전산 dataset\솔림',last_dataset_day,"pkl"),"법조치조회새창")
else : 
    fn_law = 파일명(join(r'D:\3.자산\전산 dataset\솔림',last_dataset_day,"pkl"),"법조치조회새창")
df_law = pd.read_pickle(fn_law)

# # 잘 읽어졌나 확인
df_law[df_law["사건번호"]=="2022가소582163"].계좌키.values[0][:9]

'200957567'

In [ ]:
# 구 코드 ###########
# # 법조치조회 업데이트필요하면 전산조회.ipynb에서 작업
# # cols = 채무자키, 매각사구분, 법조치구분, 관할법원, 사건번호, 청구금액법원, 관련사건번호, 매각사
# path_pkl = r'D:\1.전산,개발\3.파일서버관리\0.월별전산조회'
# ##########################################
# file_name = "법조치조회새창_20230621_0914스캔파일작업용"
# ###########################################
# # pkl 읽기
# df_law = pd.read_pickle(join(path_pkl, file_name + ".pkl"))
# # # 잘 읽어졌나 확인
# # df_law[df_law["사건번호"]=="2022가소582163"].계좌키.values[0][:9]

In [10]:
# 키매칭 후 집행문시트에 추가
for i, v in df.iterrows() :
    # 사건번호
    df_result = df_law[(df_law.관할법원== v.법원) & (df_law.사건번호== v.사건번호)].loc[:,["채무자키", "계좌키", "제소신청일", "법조치키", "주민번호"]].fillna("")
    df_result = df_result.sort_values("채무자키") # 다 채무자키인 경우 가장 앞선 채무자키를 부여하기 위해
    df_result = df_result.drop_duplicates(subset="주민번호") # 다 채무자키인 경우에도 하나만 남기기 위함
    
    # 등록 사건번호 있는 경우
    if len(df_result) == 1 : 
        df.loc[i, ["채무자키", "법조치키", "제소신청일"]] = np.concatenate(df_result[["채무자키","법조치키","제소신청일"]].values).tolist() # 제소신청일과 접수일은 차이가 있을 수 있음.. .#######
        df.loc[i, ["계좌키"]] = df_result["계좌키"].values[0][:9] # 다계좌키인경우 하나만 입력하기 위해 
    
    # 등록사건번호 없고, 관련사건번호 있는 경우
    elif len(df_result)  == 0 :
        # 법조치키는 안 불러오기
        df_result_관련 = df_law[((df_law.관련사건법원== v.법원) & (df_law.관련사건번호== v.사건번호)) | ((df_law.관련법조치관할법원== v.법원) & (df_law.관련법조치사건번호== v.사건번호))].loc[:,["채무자키", "계좌키", "제소신청일", "주민번호"]].fillna("")
        df_result_관련 = df_result_관련.sort_values("채무자키")
        df_result_관련 = df_result_관련.drop_duplicates(subset="주민번호")
        
        if len(df_result_관련) == 1 : 
            df.loc[i, ["채무자키", "제소신청일"]] = np.concatenate(df_result_관련[["채무자키","제소신청일"]].values).tolist() # 제소신청일과 접수일은 차이가 있을 수 있음.. .#######
            df.loc[i, ["계좌키"]] = df_result_관련["계좌키"].values[0][:9] # 다계좌키인경우 하나만 입력하기 위해
    
    # 중복 사건 등록 : 법조치 수정할 것    
    else :
        df.loc[i, "채무자키"] = "법조치 중복 등록"
        
df.fillna("", inplace=True)

# 엑셀에 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

### 종료건 확인 후 엑셀 다시 열기
- 전산에서 최신 채무상태 및 담당자 확인

In [ ]:
# 엑셀 수작업 후 다시 열기
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
df_집행문 = df.query('집행문==""')
total_num = len(df_집행문)
print("출력할 수 : ",total_num) # 집행문 신청용이므로 여기서는 권고나 집행문부여는 x
if total_num > 0 :
    display(df_집행문.iloc[[0,-1], cols_idx])

출력할 수 :  0


### driver 열기

In [20]:
driver, wait = con_scourt()
login()

# 집행문 신청

In [ ]:

# 대법원 조회
# 오류나면 무한대기 되어버리니 try-except하지말고 그냥 종료시키자.
cnt_success, cnt_fail = 0, 0
for i, v in df_집행문.iterrows() :
    try : 
        if v.집행문 == "" :
            # 제증명 신청-집행문:0 | 송달증명:1 | 확정증명:2 | 판결정본:3
            docu_kind = "집행문"
            증명서선택함수(docu_kind)
            
            # 법원 및 사건번호 및 확인
            select_court_event(v.법원, v.사건번호, v.피고)
            
            # 확인 버튼
            driver.find_element(By.ID, 'mf_pfwork_btn_confirm').click()
            mwt()
            
            # 오류창 확인
            오류체크() # raise ErrorMsg
            lwt()
                    

            # 발급당사자 : (0)집행,송달-당사자 / (1)확정-사건
            # 리턴값[0] : 참거짓(양수금/양수금아님)
            # 리턴값[1] : 사건명(eventname)
            문서작성결과 = 문서작성함수(0, docu_kind)
            if 문서작성결과[0] : # 정상작성
                pass 
            else : # 양수금 아니어서 제증명절차 중단해야 하는 건
                df.loc[i, ["집행문","확정","송달"]] = 문서작성결과[1] # 양수금이 아닌 경우
                cnt_fail += 1
                continue

            # 동의 및 작성완료
            lwt()
            driver.find_element(By.ID, 'mf_pfwork_wfm_elctnLwstAgre_cbx_elctnLwstAgre_input_0').click() # 동의 체크박스
            driver.find_element(By.ID, 'mf_pfwork_btn_wrtCmptn').click() # 작성완료
            mwt()
            
            # 오류창 확인
            오류체크()
                                  
            
            # 문서이상없음 및 확인
            lwt()
            check_document() 

            # 은행선택 및 납부
                # 일반적인 셀렉트로 하면 visible이 막혀서 안 됨
            bank_select_element = driver.find_element(By.ID, "mf_pfwork_sbx_vtulAcntIssuBankCd")
            bank_select = Select(bank_select_element)
            bank_select.select_by_visible_text("신한은행")
            swt()
            
            wait.until(EC.element_to_be_clickable((By.ID, "mf_pfwork_btn_lwstCstPay"))).click() # 소송비용납부
            swt()
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".w2trigger.btn_cm.pop[value='예']"))).click() # 예
            
            mwt()
            문서제출버튼클릭()
            mwt()
            certify_esc() # 일단 한 번 취소
            mwt()
            문서제출버튼클릭()
            mwt()

            # 공인인증
            try : 
                certify()
            except : 
                driver.find_element(By.ID, 'xwup_CancelButton').click() # 취소
                mwt()
                문서제출버튼클릭()
                mwt()
                certify() # 한 번 더
            
            
            lwt()

            # 계좌정보 담기 및 완료버튼
            account = driver.find_element(By.ID, 'mf_pfwork_wq_uuid_351').text # 계좌정보 담기
            wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_btn_sendLst'))).click() # 완료버튼
            swt()

            # df 수정 : 계좌번호 및 집행문 상태
            df.loc[i, "가상계좌번호"] = str(account) 
            df.loc[i, "집행문"] = "신청(미납부)"
            cnt_success += 1
            
            print(f'{i}/{total_num}, {v.채무자키}, {account}')
            
    except ErrorMsg as e : 
        cnt_fail += 1
        오류종류 = str(e)
        print(f"실패 : {i}/{total_num}, {오류종류}, {v.법원}, {v.사건번호}, {v.피고}")
        df.loc[i, ["집행문","확정","송달"]] = 오류종류
        continue
        
    except Exception as e :
        cnt_fail += 1
        print(f"실패 : {i}/{total_num}, {v.법원}, {v.사건번호}, {v.피고}")
        continue
        

# 드라이버 종료 및 엑셀 저장
driver.quit()
print(f"성공 {cnt_success}건, 실패 {cnt_fail}건")

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

 17%|█▋        | 1/6 [00:50<04:12, 50.40s/it]

20428104 56216873580080


 33%|███▎      | 2/6 [01:36<03:12, 48.03s/it]

20427733 56216873580234


 50%|█████     | 3/6 [02:21<02:19, 46.57s/it]

...

 83%|████████▎ | 5/6 [03:24<00:35, 35.81s/it]

실패 : 타지점, 부산지방법원, 2023가단302186 , 김상웅


100%|██████████| 6/6 [04:24<00:00, 44.13s/it]

20480192 56216873580610


성공 5건, 실패 1건


# 집행문 신청 후(입금 전)

## 이체~판결문목록까지 작성하기

In [ ]:
# # 새로 열어서 할 때(타지점 건까지)
# df = df_origin

In [26]:
####################################################
cost_day = str(date.today()) # 경우에 따라서 직접 입력 (20230106)
valid_word = "신청"
df_valid = df[df.집행문.str.contains(valid_word)].copy() # 집행문 발행할 목록
####################################################
cost = 450
print(f'이체건수 : {len(df_valid)}')



# 등록일은 안 써도 됨
# 1) 이체용 df만들기 : 집행문 신청 후 가상계좌번호 받고 난 뒤
dict_cols = {"이체" : ["입금은행", "입금계좌", "고객관리성명", "입금액", "출금통장표시내용", "입금통장표시내용", "입금인코드", "비고", "업체사용key"],
            "법조치전체" : ["법조치키", "채무자키","계좌키","대상자명","담당자","법조치구분","법조치세부","법조치상태","관할법원","사건번호","접수일","법조치메모"],
            "메모" : ["메모키","채무자키","주민번호","매각사","매각사구분","원채권사","메모내용","메모구분","등록일자","등록유저","공지여부","메모색상","약속일자","약속금액","약속구분","약속이행"],
            "법비용" : ["법비용키","채무자키","계좌키","법조치키","법비용일","법비용구분","법비용내역","법비용메모","기타비"],
            "판결문" : ["발급일자","채무자키","채무자","법원","사건번호","재도_승계_등등"]
            }

content_display=[]
names = []
for i in df_valid.피고.str.replace("주식회사|유한회사|법인|대표|보증인?|\b[주유]\b|\(주|주\)|외\s?\d.*", "", regex=True) :
    
    # 법조치 대상자명용
    i = re.sub("[^가-힣a-zA-Z]", " ", i).strip()
    sign = re.search("\s", i)
    if sign :
        new_i = i[:sign.start()]
    else :
        new_i = i
    if len(new_i) >= 2 : 
        names.append(new_i)
    else : names.append(i)
    # 이체 화면표시용
    if len(new_i) > 3 : content = new_i[:3]+" 제증명"
    elif len(new_i) >= 2 :
        content = new_i + " 제증명"
    elif len(i) > 3 : 
        content = i[:3] + " 제증명"
    else :
        content = i + " 제증명"
    content_display.append(content)

df_transfer = pd.DataFrame(None, columns=dict_cols["이체"])
df_transfer["입금계좌"] = df_valid.가상계좌번호
df_transfer["출금통장표시내용"] = content_display
# 단일 값은 뒤에(그래야 모두 채워지니)
df_transfer["입금은행"] = "088"
df_transfer["입금액"] = cost

# 4) 법비용 #법조치키는 전산에서 조회해서 수동update
######## 수정일, 수정유저(나) 검색 셀추가 및 필터링 ########
######## 검색 결과 정렬이 바뀌니, 엑셀파일과 검색결과의 정렬 맞춘 후 복붙 ########
df_cost = pd.DataFrame(None, columns=dict_cols["법비용"])
df_cost["채무자키"] = df_valid.채무자키
df_cost["계좌키"] = df_valid.계좌키
df_cost["법조치키"] = df_valid.법조치키
df_cost["법비용일"] = cost_day
df_cost["법비용구분"] = "기본비용"
df_cost["법비용내역"] = "초도부여-신청비용" # 집행문부여
df_cost["집행문발급비"] = cost


# df_valid 재작성 : 이행권고,집행문부여 포함 -------------------------------------------------------------------------------------------------
if 이행권고출력 and 집행문부여출력 : 
    valid_word = valid_word + "|이행권고|집행문부여"
elif 집행문부여출력 :
    valid_word = valid_word + "|집행문부여"
elif 이행권고출력 :
    valid_word = valid_word + "|이행권고"

df_valid2 = df[df.집행문.str.contains(valid_word)].copy() # 집행문 발행할 목록 ##################################

names = []
for i in df_valid2.피고.str.replace("주식회사|유한회사|법인|대표|보증인?|\b[주유]\b|\(주|주\)|외\s?\d.*", "", regex=True) :
    
    # 법조치 대상자명용
    i = re.sub("[^가-힣a-zA-Z]", " ", i).strip()
    sign = re.search("\s", i)
    if sign :
        new_i = i[:sign.start()]
    else :
        new_i = i
    if len(new_i) >= 2 : 
        names.append(new_i)
    else : names.append(i)
    

# 2) 법조치전체 : df에 채무자키 등 추가 후
df_legalaction = pd.DataFrame(None, columns=dict_cols["법조치전체"])
df_legalaction["법조치키"] = df_valid2.법조치키
df_legalaction["채무자키"] = df_valid2.채무자키
df_legalaction["계좌키"] = df_valid2.계좌키
df_legalaction["대상자명"] = names
df_legalaction["관할법원"] = df_valid2.법원
df_legalaction["사건번호"] = df_valid2.사건번호
df_legalaction["접수일"] = df_valid2.제소신청일
df_legalaction["법조치구분"] = np.where(df_valid2.집행문.str.contains("집행문부여"), "집행문부여의소", "본안소송")
df_legalaction["법조치세부"] = "전자소송"
df_legalaction["법조치상태"] = "법조치완료"

# 3) 메모
date.today()
df_memo = pd.DataFrame(None, columns=dict_cols["메모"])
df_memo["채무자키"] = df_valid2.채무자키
# df_memo["메모내용"] = df_valid2.apply(lambda row: f"{row['사건번호']} 집행문 발급(이행권고)" if row["집행문"] == "이행권고" else f"{row['사건번호']} 집행문 발급", axis=1)
df_memo["메모내용"] = np.where(df_valid2.집행문.str.contains("신청"), df_valid2.사건번호+" 집행문 발급",
    np.where(df_valid2.집행문.str.contains("이행권고"), df_valid2.사건번호+" 집행문 발급(이행권고)", df_valid2.사건번호+" 집행문부여의소 판결문정본 발급"))
df_memo["등록일자"] = str(date.today()) # str(date.today() + timedelta(days=3)) # 금요일에 작업하고, 월요일에 등록하는 경우 
df_memo["메모구분"] = "기타"
df_memo["등록유저"] = "전병만"

# 5) 판결문
df_list = pd.DataFrame(None, columns=dict_cols["판결문"])
df_list["발급일자"] = str(date.today())
df_list["채무자키"] = df_valid2.채무자키
df_list["채무자"] = df_valid2.피고
df_list["법원"] = df_valid2.법원
df_list["사건번호"] = df_valid2.사건번호
df_list["사건구분"] = ""
df_list["담당자"] = ""
df_list["재도_승계_등등"] = np.where(df_valid2.집행문.str.contains('집행문|권고'), df_valid2.집행문, "")

print(f'사건건수 : {len(df_valid2)}')

이체건수 : 12
사건건수 : 12


#### 엑셀 작성

In [27]:
dict_sheet_df = {"이체": df_transfer, "법조치전체":df_legalaction,"메모":df_memo,"법비용":df_cost,"판결문":df_list}

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    for sheetname, df_name in dict_sheet_df.items():
        df_name.to_excel(writer, sheet_name = sheetname, index=False)

# 입금
- 만료일 10-27

# 확정증명원/송달증명원 신청
- (250212)요새 좀 이상하니까 출력전에 신청하자. 확정건이 수동발급으로 진행되는 경우 많은듯.

In [33]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

In [34]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
valid_word = "신청|발급" # 이행권고는 송달,확정 필요없음. 집행문부여는 필요함
if 집행문부여출력 :
    valid_word = valid_word + "|집행문부여"

df_증명원 = df.query('집행문.str.contains(@valid_word)')
print("신청할 사건 수 : ", len(df_증명원))
df_증명원.iloc[[0,-1],cols_idx]

신청할 사건 수 :  12


,법원,사건번호,확정일자,피고,집행문,확정,송달
0,광주지방법원,2024가소612043,2025.02.08,남성일,신청(미납부),,
11,광주지방법원,2023가소590337,2024.06.08,강문수,신청(미납부),,


In [ ]:
# 건당 1분씩 소요
cnt_success_fix, cnt_success_del, cnt_fail = 0, 0, 0
for i, v in tqdm(df_증명원.iterrows(), total=len(df_증명원)) :
    try : # 송달에서 오류나는 경우는 거의 없으므로 송달먼저하고, try 한번만 하자
        if v.송달 == "" :
            fix_delivery("송달", v)
            df.loc[i, "송달"] = "신청" # df수정
            cnt_success_del += 1

        if v.확정 == "" :
            fix_delivery("확정", v)
            df.loc[i, "확정"] = "신청" # df수정
            cnt_success_fix += 1
    except :
        cnt_fail += 1
        continue

print(f"신청성공[확정증명원 : {cnt_success_fix}건, 송달증명원 : {cnt_success_del}건], 실패 {cnt_fail}건")
# 엑셀 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

100%|██████████| 12/12 [03:42<00:00, 18.54s/it]

신청성공[확정증명원 : 0건, 송달증명원 : 2건], 실패 11건


In [117]:
df[(df.집행문.str.contains("신청|집행문부여")) & ((df.송달=="")|(df.확정==""))][["법원","사건번호","피고","집행문","확정","송달"]]

,법원,사건번호,피고,집행문,확정,송달


# 출력

##### 엑셀 및 드라이버 열기

In [ ]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
print("전체건수",len(df))
valid_word = "신청"
if 이행권고출력 and 집행문부여출력 : 
    valid_word = valid_word + "|이행권고|집행문부여"
elif 집행문부여출력 :
    valid_word = valid_word + "|집행문부여"
elif 이행권고출력 :
    valid_word = valid_word + "|이행권고"
################ 타지점건은 비고에 지점 작성. 출력은 따로 해야 하니 비고가 공백인것만
df_출력 = df.query('비고=="" and 집행문.str.contains(@valid_word) or 확정=="신청" or 송달=="신청"')
print("출력대상", len(df_출력))
df_출력.iloc[[0,-1],cols_idx]

전체건수 13
출력대상 6


,법원,사건번호,확정일자,피고,집행문,확정,송달
6,광주지방법원,2024가소569781,2025.02.01,추순자,신청(납부),신청,신청
11,광주지방법원,2024가소562964,2025.01.17,박승용,신청(납부),신청,신청


In [182]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

##### 프리턴 설정 확인 not pdf, 이행권고 미확인 문서 일괄확인할 것

In [183]:
class CustomBreakException(Exception): # 내for문에서라도 해당 지점에서 모든 반복문을 종료하기 위한 클래스(break)
    pass

cnt_success, cnt_success_fix, cnt_success_del, cnt_success_판결정본 = 0,0,0,0
cnt_fail, cnt_fail_fix, cnt_fail_del, cnt_fail_판결정본 = 0,0,0,0

try : 
    for i, v in tqdm(df_출력.iterrows(), total=len(tqdm)) :
        ###################
        #if v.번호 < 2 :
        ###################
        # 제증명에서 발급할 것이 하나라도 있는 사건번호를 검색(조회)
        if (re.search("신청|집행문부여|이행권고",v.집행문)) or (v.확정 == "신청") or (v.송달 == "신청") : 
            
            # 1) 집행문부여, 이행권고 출력 : 송달문서 정(등)본발급
            if re.search("집행문부여|이행권고", v.집행문) and re.search("발급", v.집행문)==None: 
                try : 
                    # 발급조회, 이행권고와 같음
                    이행권고()
                    driver.find_element(By.ID, "mf_pfwork_rad_choice_input_1").click() # 사건번호 radio
                    swt()
                    select_court_event(v.법원, v.사건번호, name="", 발급관련=True)
                    driver.find_element(By.ID, "mf_pfwork_btn_search").click() # 조회버튼 클릭
                    mwt()
                    
                    # 출력
                    권고_result = print_이행권고()
                    df.loc[i, "집행문"] = 권고_result + "_" + v.집행문
                    if 권고_result == '발급' : cnt_success_판결정본 += 1
                except : 
                    print(v.사건번호, v.집행문, "판결정본 출력 중 에러")
                    cnt_fail_판결정본 += 1
                    
            
            # 2-1) 집행문, 송달확정 발급 건 사건조회
            if re.search("신청", v.집행문) or (v.확정 == "신청") or (v.송달 == "신청") : 
                temp_cnt = 0
                max_attempts = 3
                for attempt in range(max_attempts) : # 사건조회에서 에러나면 
                    try : 
                        출력_사건번호조회(v.법원, v.사건번호)
                        break # 오류없음
                        # 오류가 생길 가능성보다 조회 결과 없을 가능성..
                        # 조회 결과 없는 경우를 처리하고 싶으면 오류검사함수 활용
                        
                    except Exception as e :
                        temp_cnt += 1
                        if attempt < max_attempts :
                            print(f"조회화면 에러 재시도 : {temp_cnt}회")
                        else:
                            print("최대 시도 횟수를 초과하여 모든 작업 중단")
                            break
                        
                        # 오류시 모든 팝업 닫고 새로고침
                        close_all_popups_and_refresh(driver)
            
            # 출력할 거 없으면 다음 행으로
            else : continue
            
            # 2-2) 집행문, 송달확정 출력 - 집행문 에러나도 확정, 송달 출력되게 하나씩 예외처리
            # 2-2-1) 집행문
            try : 
                if re.search("신청",v.집행문) : # v.집행문은 str임... 미납부 > 납부 수정 깜박하는 경우가 많아서.. '==신청(납부)'에서 바꿈
                    exe_result = print_execution("집행문(정본포함)") 
                    df.loc[i, "집행문"] = exe_result
                    if exe_result == "발급" : cnt_success += 1
            except : 
                print(v.사건번호, "집행문 출력 중 에러")
                cnt_fail += 1
                
                # 에러시 화면 클리어하고 사건번호 재조회(아래 남은 코드라도 실행)
                close_all_popups_and_refresh(driver)
                출력_사건번호조회(v.법원, v.사건번호)
                
            # 2-2-2) 송달
            try : 
                if v.송달 == "신청" : # 송달도 발급해야 하면 연속 발급
                    del_result = print_execution("송달증명")
                    df.loc[i, "송달"] = del_result
                    if del_result == "발급" : cnt_success_del += 1
            except : 
                print(v.사건번호, "송달증명 출력 중 에러")
                cnt_fail_del += 1
                
                # 에러시 화면 클리어하고 사건번호 재조회(아래 남은 코드라도 실행)
                close_all_popups_and_refresh(driver)
                출력_사건번호조회(v.법원, v.사건번호)
                
            # 2-2-3) 확정
            try : 
                if v.확정 == "신청" :
                    fix_result = print_execution("확정증명")
                    df.loc[i, "확정"] = fix_result
                    if fix_result == "발급" : cnt_success_fix += 1
            except : 
                print(v.사건번호, "확정증명 출력 중 에러")
                cnt_fail_fix += 1
                
                close_all_popups_and_refresh(driver) # 마지막이라 사건번호 입력할 필요는 없음.
                    
except CustomBreakException:
    print("에러로 모든 반복문 종료")
    
finally :
    print(f"출력성공[집행문:{cnt_success}건, 확정:{cnt_success_fix}건, 송달:{cnt_success_del}건, 판결정본:{cnt_success_판결정본}건]")
    print(f"출력실패[집행문:{cnt_fail}건, 확정:{cnt_fail_fix}건, 송달:{cnt_fail_del}건, 판결정본:{cnt_fail_판결정본}건]")
    # 드라이버 종료 및 엑셀 저장
    driver.quit()
# 엑셀 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

# 이행권고 기발급건(재발급요청)은 메모와 판결문 목록에서는 삭제해줘야

2024가소562964 집행문 출력 중 에러
출력성공[집행문:4건, 확정:6건, 송달:6건, 판결정본:0건]
출력실패[집행문:1건, 확정:0건, 송달:0건, 판결정본:0건]


# 발급 및 스캔 후 : 스캔파일
- 인쇄 순서 엉킨 경우에는 순서열 만들어서 정렬 후 읽기

In [184]:
# 스캔파일 목록을 df_scan에 추가하기
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문", dtype=dtype).fillna("")
df = df_origin.copy()
df_valid = df[df.집행문.str.contains('발급')].copy().reset_index(drop=True) # 실제 작업할 영역 불러오기 # 240415 이행권고 때문에 =='발급'에서 포함으로 바꿈
# df_valid = df.iloc[:10,:].copy() # 실제 작업할 영역 불러오기
##################################################
if 보조pc : 
    path = r"\\192.168.10.83\scan"
else : 
    path = r"D:\scan"
file_list = read_filelist(path) # 탐색기 정렬과 무관하게 이름 오름차순으로 정렬된 리스트 반환
df_valid["file"] = file_list

In [185]:
# df_valid 한행씩 불러와서 파일명 바꾸기
success, fail = [], []
for i, v in df_valid.iterrows() :
    채무자명 = re.sub(':', ' ', v.피고) # '파일명 사용 불가 기호 삭제. ex(변경전 : ~~~) ' 
    ext = os.path.splitext(v.file)[1]
    if re.search("이행권고", v.집행문) : 
        new_name = "_".join([str(int(v.채무자키)), 채무자명, v.사건번호, "이행권고"]) + ext
    elif re.search("집행문부여", v.집행문) : 
        new_name = "_".join([str(int(v.채무자키)), 채무자명, v.사건번호, "집행문부여의소"]) + ext
    else : 
        new_name = "_".join([str(int(v.채무자키)), 채무자명, v.사건번호]) + ext
    try :
        os.rename(join(path, v.file), join(path, new_name))
        success.append([path, v.file, new_name])
    except Exception as e :
        fail.append([path, v.file, new_name, e.__class__])
        continue
print(f"총 {len(df_valid)}개 중, {len(success)}개 파일명 변경, {len(fail)}개 파일명 변경 실패")

총 10개 중, 10개 파일명 변경, 0개 파일명 변경 실패


In [1]:
# 옛날꺼
# path = r"D:\scan\새 폴더"
# p_e = re.compile("\d{4}[가-힣]{1,3}\d+")
# df_result = pd.DataFrame(columns=["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"])
# for f in os.listdir(path) :
#     stem, ext = os.path.splitext(f)
#     e_name = p_e.search(stem).group()
#     # drop_duplicates(subset=["채무자키"], keep='first').
#     keys = df_law[(df_law.관련사건번호== e_name) |(df_law.사건번호== e_name)].loc[:,["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"]]
#     df_result = pd.concat([df_result, keys])
    
# df_result    

#### 법조치 등록 후 법비용에 법조치키 수동 입력
- 법조치새창 > 사건번호 멀티검색 > "법조치시트"에서 사건번호 가져와 > 수정유저 컬럼 추가 후 '자신'으로 필터링
- 순서 다르므로 엑셀과 전산프로그램 둘다 채무자키로 정렬 후 "법비용시트"에 복붙